# Introduction

This notebook provides you an opportunity to demonstrate proficiency in meeting course learning goals by applying a support vector machine to solve a classification problem using widely-used ML libraries and an ML workflow.


# Mine Detection (Revisited)

In this notebook, you will revisit [a previously seen classification problem](https://www.kaggle.com/code/bakosy/cs-513-notebook-4-classification-with-perceptrons), and see if you can build a better classification model that can predict whether or not a sonar signature is from a mine or a rock.

<div class="alert alert-block alert-warning">
<b>Tip:</b> We suggest reviewing your Notebook 4: Classification with Perceptrons.
</div>

We'll use a version of the [sonar data set](https://www.openml.org/search?type=data&sort=runs&id=40&status=active) by Gorman and Sejnowski. Take a moment now to [reacquaint yourself with the subject matter of this data set](https://datahub.io/machine-learning/sonar%23resource-sonar), and look at the details of the version of this data set, [Mines vs Rocks, hosted on Kaggle](https://www.kaggle.com/datasets/mattcarter865/mines-vs-rocks).

Similar to [a previous notebook](https://www.kaggle.com/code/bakosy/cs-513-notebook-4-classification-with-perceptrons), this notebook expects each student to implement the ML workflow steps. We will get you started by providing the first step, loading the data, and providing some landmarks and tips below. Your process should demonstrate:

1. Loading the data
2. Exploring the data
3. Preprocessing the data
4. Preparing the training and test sets
5. Creating and configuring a sklearn.svm.SVC
6. Training the SVM
7. Validating and Testing the SVM
8. Demonstrating making predictions
9. Evaluate (and Improve) the results

Can you train a classifier that can predict whether a sonar signature is from a mine or a rock? "Three trained human subjects were each tested on 100 signals, chosen at random from the set of 208 returns used to create this data set. Their responses ranged between 88% and 97% correct." Can your classifier outperform the human subjects?

Most importantly, how does the performance of the SVM classifier compare to the perceptron results observed in [Notebook 4](https://www.kaggle.com/code/bakosy/cs-513-notebook-4-classification-with-perceptrons)?



## Step 1: Load the Data

The notebook comes pre-bundled with the [Mines vs Rocks data set](https://www.kaggle.com/datasets/mattcarter865/mines-vs-rocks). Our first step is to create a pandas DataFrame from the CSV file. Note that the CSV file has no header row. Loading the CSV file into a DataFrame will make it easy for us to explore the data, preprocess it, and split it into training and test sets.


In [1]:
import pandas as pd

sonar_csv_path = "../input/mines-vs-rocks/sonar.all-data.csv"
sonar_data = pd.read_csv(sonar_csv_path, header=None)



We now have a pandas DataFrame encapsulating the sonar data, and can proceed with our data exploration.

## Step 2: Explore the Data

Let's take a look at the data using the `.head()` method. This will give us the dimensions, and a good look at the first five rows.

In [2]:
sonar_data.head()

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,R
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,R
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,R
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,R
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,R


We see we have 61 columns, which is what we expect based on the documentation on the kaggle website where the data came from. The first 60 columns are the features, and the last one is the labels. The first five are all "R" for rock, but we would expect some others would be "M" for mine. 

Next, let's use the `.describe` method to get some statistics on the features to see the range they occupy. We first set some pandas options so it will display all of the columns instead of hiding the middle ones.

In [3]:
pd.set_option('display.max_columns',None)
sonar_data.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59
count,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000
mean,0.029164,0.038437,0.043832,0.053892,0.075202,0.104570,0.121747,0.134799,0.178003,0.208259,0.236013,0.250221,0.273305,0.296568,0.320201,0.378487,0.415983,0.452318,0.504812,0.563047,0.609060,0.624275,0.646975,0.672654,0.675424,0.699866,0.702155,0.694024,0.642074,0.580928,0.504475,0.439040,0.417220,0.403233,0.392571,0.384848,0.363807,0.339657,0.325800,0.311207,0.289252,0.278293,0.246542,0.214075,0.197232,0.160631,0.122453,0.091424,0.051929,0.020424,0.016069,0.013420,0.010709,0.010941,0.009290,0.008222,0.007820,0.007949,0.007941,0.006507
std,0.022991,0.032960,0.038428,0.046528,0.055552,0.059105,0.061788,0.085152,0.118387,0.134416,0.132705,0.140072,0.140962,0.164474,0.205427,0.232650,0.263677,0.261529,0.257988,0.262653,0.257818,0.255883,0.250175,0.239116,0.244926,0.237228,0.245657,0.237189,0.240250,0.220749,0.213992,0.213237,0.206513,0.231242,0.259132,0.264121,0.239912,0.212973,0.199075,0.178662,0.171111,0.168728,0.138993,0.133291,0.151628,0.133938,0.086953,0.062417,0.035954,0.013665,0.012008,0.009634,0.007060,0.007301,0.007088,0.005736,0.005785,0.006470,0.006181,0.005031
min,0.001500,0.000600,0.001500,0.005800,0.006700,0.010200,0.003300,0.005500,0.007500,0.011300,0.028900,0.023600,0.018400,0.027300,0.003100,0.016200,0.034900,0.037500,0.049400,0.065600,0.051200,0.021900,0.056300,0.023900,0.024000,0.092100,0.048100,0.028400,0.014400,0.061300,0.048200,0.040400,0.047700,0.021200,0.022300,0.008000,0.035100,0.038300,0.037100,0.011700,0.036000,0.005600,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000800,0.000500,0.001000,0.000600,0.000400,0.000300,0.000300,0.000100,0.000600
25%,0.013350,0.016450,0.018950,0.024375,0.038050,0.067025,0.080900,0.080425,0.097025,0.111275,0.129250,0.133475,0.166125,0.175175,0.164625,0.196300,0.205850,0.242075,0.299075,0.350625,0.399725,0.406925,0.450225,0.540725,0.525800,0.544175,0.531900,0.534775,0.463700,0.411400,0.345550,0.281400,0.257875,0.217575,0.179375,0.154350,0.160100,0.174275,0.173975,0.186450,0.163100,0.158900,0.155200,0.126875,0.094475,0.068550,0.064250,0.045125,0.026350,0.011550,0.008425,0.007275,0.005075,0.005375,0.004150,0.004400,0.003700,0.003600,0.003675,0.003100
50%,0.022800,0.030800,0.034300,0.044050,0.062500,0.092150,0.106950,0.112100,0.152250,0.182400,0.224800,0.249050,0.263950,0.281100,0.281700,0.304700,0.308400,0.368300,0.434950,0.542500,0.617700,0.664900,0.699700,0.698500,0.721100,0.754500,0.745600,0.731900,0.680800,0.607150,0.490350,0.429600,0.391200,0.351050,0.312750,0.321150,0.306300,0.312700,0.283500,0.278050,0.259500,0.245100,0.222550,0.177700,0.148000,0.121350,0.101650,0.078100,0.044700,0.017900,0.013900,0.011400,0.009550,0.009300,0.007500,0.006850,0.005950,0.005800,0.006400,0.005300
75%,0.035550,0.047950,0.057950,0.064500,0.100275,0.134125,0.154000,0.169600,0.233425,0.268700,0.301650,0.331250,0.351250,0.386175,0.452925,0.535725,0.659425,0.679050,0.731400,0.809325,0.816975,0.831975,0.848575,0.872175,0.873725,0.893800,0.917100,0.900275,0.852125,0.735175,0.641950,0.580300,0.556125,0.596125,0.593350,0.556525,0.518900,0.440550,0.434900,0.424350,0.387525,0.384250,0.324525,0.271750,0.231550,0.200375,0.154425,0.120100,0.0685

The first thing we notice about this data is that it is not normalized. The minimum and maximum values are different for each feature, which can lead to features with larger numbers being misconstrued as more important by the machine learning algorithm. It is important to normalize this later on.

Now that we have a good understanding of our features space, let's take another look at the labels and their distribution. We know that the labels are in column 60. We will use the `.value_counts()` method to see the distribution of values in the column.

In [4]:
label_distribution = sonar_data[60].value_counts()
print(label_distribution)
print("Total objects: " + str(len(sonar_data)))

M    111
R     97
Name: 60, dtype: int64
Total objects: 208


There are 111 data objects labeled "mine", and 97 data objects labeled "rock". This is not evenly distributed, so balancing the label weights during training might be advantageous. This also means there are 208 total data objects, which we confirm with the len() function above.

The next step is to make sure there are no null values I need to resolve. This is very simple with pandas.

In [5]:
sonar_data.isnull().values.any()

False

There are no null values, so it looks like everything is good to go. We are ready to start preprocessing the data.

## Step 3: Preprocess the Data

First we import `StandardScaler` from sklearn, which normalizes each feature by its z-score (z = (x-u) / s, where u is the mean  and s is the standard deviation of each feature). Next, we split the features (X) from the labels (y) by slicing the pandas dataframe. Then, we initialize the scaler, scale it, and put the data back into a dataframe so we can use the `describe()` method.

In [6]:
from sklearn.preprocessing import StandardScaler

X = sonar_data.iloc[:,:60]
y = sonar_data.iloc[:,60]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled = pd.DataFrame(X_scaled)

X_scaled.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59
count,2.080000e+02,2.080000e+02,2.080000e+02,2.080000e+02,2.080000e+02,2.080000e+02,2.080000e+02,2.080000e+02,2.080000e+02,2.080000e+02,2.080000e+02,208.000000,2.080000e+02,2.080000e+02,2.080000e+02,2.080000e+02,2.080000e+02,2.080000e+02,2.080000e+02,208.000000,2.080000e+02,2.080000e+02,2.080000e+02,2.080000e+02,2.080000e+02,2.080000e+02,2.080000e+02,2.080000e+02,2.080000e+02,2.080000e+02,2.080000e+02,2.080000e+02,2.080000e+02,2.080000e+02,2.080000e+02,2.080000e+02,2.080000e+02,2.080000e+02,2.080000e+02,2.080000e+02,2.080000e+02,2.080000e+02,2.080000e+02,2.080000e+02,2.080000e+02,2.080000e+02,2.080000e+02,2.080000e+02,2.080000e+02,2.080000e+02,2.080000e+02,2.080000e+02,2.080000e+02,2.080000e+02,2.080000e+02,2.080000e+02,2.080000e+02,2.080000e+02,2.080000e+02,2.080000e+02
mean,1.708035e-17,6.832142e-17,-1.195625e-16,1.622634e-16,-1.793437e-16,2.049643e-16,1.024821e-16,3.416071e-17,-3.757678e-16,3.416071e-17,-3.416071e-17,0.000000,6.832142e-17,-2.562053e-17,-5.551115e-17,1.281027e-17,2.604754e-16,-8.540177e-18,-9.394195e-17,0.000000,4.440892e-16,-1.195625e-16,-3.074464e-16,3.416071e-17,-7.771561e-16,5.636517e-16,-2.647455e-16,-8.540177e-17,1.281027e-16,1.537232e-16,2.562053e-17,1.494531e-16,-5.978124e-17,1.708035e-17,1.024821e-16,1.195625e-16,2.732857e-16,-1.216975e-16,5.978124e-17,2.732857e-16,-1.494531e-16,-1.024821e-16,8.540177e-17,3.074464e-16,-3.416071e-17,1.708035e-16,1.366428e-16,2.391250e-16,-1.366428e-16,-3.074464e-16,3.416071e-17,1.024821e-16,3.416071e-17,-1.451830e-16,2.775558e-17,-2.391250e-16,3.416071e-17,-1.110223e-16,1.345078e-16,7.686159e-17
std,1.002413e+00,1.002413e+00,1.002413e+00,1.002413e+00,1.002413e+00,1.002413e+00,1.002413e+00,1.002413e+00,1.002413e+00,1.002413e+00,1.002413e+00,1.002413,1.002413e+00,1.002413e+00,1.002413e+00,1.002413e+00,1.002413e+00,1.002413e+00,1.002413e+00,1.002413,1.002413e+00,1.002413e+00,1.002413e+00,1.002413e+00,1.002413e+00,1.002413e+00,1.002413e+00,1.002413e+00,1.002413e+00,1.002413e+00,1.002413e+00,1.002413e+00,1.002413e+00,1.002413e+00,1.002413e+00,1.002413e+00,1.002413e+00,1.002413e+00,1.002413e+00,1.002413e+00,1.002413e+00,1.002413e+00,1.002413e+00,1.002413e+00,1.002413e+00,1.002413e+00,1.002413e+00,1.002413e+00,1.002413e+00,1.002413e+00,1.002413e+00,1.002413e+00,1.002413e+00,1.002413e+00,1.002413e+00,1.002413e+00,1.002413e+00,1.002413e+00,1.002413e+00,1.002413e+00
min,-1.206158e+00,-1.150725e+00,-1.104253e+00,-1.036115e+00,-1.236093e+00,-1.600493e+00,-1.921613e+00,-1.522110e+00,-1.443689e+00,-1.468833e+00,-1.564472e+00,-1.621794,-1.812688e+00,-1.641094e+00,-1.547347e+00,-1.560974e+00,-1.448751e+00,-1.589952e+00,-1.769499e+00,-1.898502,-2.168994e+00,-2.359782e+00,-2.366740e+00,-2.719680e+00,-2.666087e+00,-2.568134e+00,-2.668895e+00,-2.813072e+00,-2.618893e+00,-2.359606e+00,-2.137348e+00,-1.873982e+00,-1.793647e+00,-1.656081e+00,-1.432336e+00,-1.430241e+00,-1.373417e+00,-1.418414e+00,-1.453707e+00,-1.680436e+00,-1.483614e+00,-1.620067e+00,-1.778046e+00,-1.609948e+00,-1.303898e+00,-1.202190e+00,-1.411667e+00,-1.468270e+00,-1.447799e+00,-1.498229e+00,-1.341343e+00,-1.313126e+00,-1.449472e+00,-1.364897e+00,-1.229092e+00,-1.366868e+00,-1.302971e+00,-1.185113e+00,-1.271603e+00,-1.176985e+00
25%,-6.894939e-01,-6.686781e-01,-6.490624e-01,-6.359298e-01,-6.703975e-01,-6.367565e-01,-6.626732e-01,-6.400918e-01,-6.856590e-01,-7.232644e-01,-8.064569e-01,-0.835483,-7.621827e-01,-7.398484e-01,-7.591598e-01,-7.849821e-01,-7.988564e-01,-8.058388e-01,-7.993884e-01,-0.810706,-8.139075e-01,-8.514605e-01,-7.883456e-01,-5.530684e-01,-6.123656e-01,-6.578782e-01,-6.947312e-01,-6.730211e-01,-7.442437e-01,-7.698181e-01,-7.444604e-01,-7.410570e-01,-7.734584e-01,-8.048124e-01,-8.247161e-01,-8.748025e-01,-8.511364e-01,-7.784131e-01,-7.644914e-01,-6.999700e-01,-7.390302e-01,-7.093139e-01,-6.587522e-01,-6.557863e-01,-6.793257e-01,-6.891506

Each feature is now normalized with its own z-score, which represents how far away each data point is (in terms of standard deviations) from the mean. Each is centered on 0, where negative numbers represent points below the mean and positive numbers represent points above. The table above shows this, so we are good to go.

## Step 4: Prepare the Training and Test Data Sets

We will use `train_test_split` from sklearn to split our data into training and testing sets. It splits our data into four different variables, which are described below.

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=0)

Now, the data is split into features (labeled "X") for training and testing, and labels (labeled "y") for training and testing. I chose to hold back 20% of the data for testing, because that is a common value to use for the hold-out method. I also set the random state to 0, so my results are reproducible. 

## Step 5: Instantiate and Configure an SVM

Here we start by importing the `SVC` class from sklearn, and setting some hyperparameters. We start with the default settings, where C is the regularization parameter, the kernel used is rbf (radial basis function), gamma is the kernel coefficient used ('scale' calculates it as 1 / (number of features * feature variance)), tol is the stopping criteria for when the weights are not changing by much anymore (indication that a minimum has been reaching in the loss function), class_weight is an option to set weights for the loss function for different classes (balanced will create weights inversely proportional to their frequencies), max_iter is the max number of iterations through the training set it will use before stopping (-1 sets it to no limit), and random_state allows you to save the random number generation used for shuffling the data so results are reproducible (None will make the shuffle random each time, any integer will represent a random pattern). There are other parameters that could be set, but in class these parameters have been shown to be the most important.

In [8]:
from sklearn.svm import SVC

svm_model = SVC(C=1.0, kernel='rbf', 
                gamma='scale', 
                tol=0.001, class_weight='balanced', 
                max_iter=-1, random_state=0)



We have now initiated our SVM model!

## Step 6: Train the SVM

Training the model is easy with this implementation. All we need to do is call the `fit` method and provide the training features and labels. 

In [9]:
svm_model.fit(X_train,y_train)


SVC(class_weight='balanced', random_state=0)

With this run, we are now ready to use our trained model and examine the accuracy.

## Step 7: Validate and Test the SVM

We will call on the `score` method and use both the training data and the testing data. This will give us the accuracy the model achieved while training, and how well it does on data it has never seen before in the testing set.


In [10]:
print("Training accuracy: " + str(svm_model.score(X_train,y_train)))
print("Testing accuracy: " + str(svm_model.score(X_test,y_test)))


Training accuracy: 0.9939759036144579
Testing accuracy: 0.8333333333333334


We have a very high training accuracy score (99.4%), but a low testing score (83.3%). This may be an indication of over-fitting in our model.

## Step 8: Demonstrate Making Predictions

It is time to show what the trained model can do. I created a random array of data that has 60 values, all between 0 and 1, just like the true features. By plugging these features into the model and using the `predict` method, it will print a prediction. 

In [11]:
import random

random_features = pd.DataFrame([random.uniform(0, 1) for i in range(60)]).values.reshape(1,-1)
#random_features = sonar_data.iloc[24,:60].values.reshape(1,-1)

prediction = svm_model.predict(random_features)
print(prediction)

['M']


Here it predicts "M", which means a mine. I also commented out a line that takes the 24th row of the original dataframe and plugs that back into the model. This was just a test to see a different result. If you uncomment it and run the cell, the model will predict an "R", which means rock.

## Step 9: Evaluate (and Improve?)

The current testing accuracy is 83.3%. This model is currently underperforming and is not a good classifier. It performed worse than trained humans, who had accuracies from 88% to 97%. We have a high training accuracy, and a low testing accuracy, which indicates overfitting is likely.

There are 60 features, which can lead to the perceptron trying to fit the noise instead of the pattern. A dimension reduction technique like PCA could be useful. I think decreasing the regularization parameter "C" would help the model not overfit the data, but I will try increasing it as well. This parameter is used to introduce penalties to the learning algorithm, so it doesn't become overly complicated in exchange for less accurate training results. Experimenting with different kernels such as poly or linear might also yield a better decision boundary. It all depends on the data, so we must experiment.

Let's take another crack at the model. I adjusted each of the hyperparameters mentioned above, and this combination provides the best accuracy. 

In [12]:
svm_model = SVC(C=4.2, kernel='rbf', 
                gamma='scale', degree=3,
                tol=0.001, class_weight='balanced', 
                max_iter=-1, random_state=0)

svm_model.fit(X_train,y_train)
print("Training accuracy: " + str(svm_model.score(X_train,y_train)))
print("Testing accuracy: " + str(svm_model.score(X_test,y_test)))

Training accuracy: 1.0
Testing accuracy: 0.9285714285714286


We have achieved 92.9% accuracy on our testing set. That is great! Our model now performs right in the middle of the average human accuracy (88% to 97%). I increased the regularization parameter "C" to 4.2, which allows the model to get a bit more complicated. Using the poly kernel, I was able to get about 90.5% accuracy, but ultimately went back to the rbf kernel. No other kernels performed as well. I played around with the other settings, but ultimately the default for each was the best.

The training accuracy is 100%, which worries me. I was able to get both accuracies around the 88% mark with one set of hyperparameters, but because I could not get better than that I changed back. Our model may still be overfitting to this data set. If I were to continue this experiment, I would do k-fold cross validation with the data to make sure the accuracies the model performs at are seen even when training and testing on different sets. I also think I would use a dimension reduction technique like PCA. I will not experiment with that here because it is beyond the scope of the workbook.

## Conclusion


In this workbook I explored the sonar dataset again, used the sklearn SVM to train and test a model that predicts whether readings indicate solid rock or a mine, and tried to improve the model accuracy by tuning hyperparameters. I learned how the standard scaler function works from sklearn. I wonder if different scaling techniques influence the performance of the model? Before we did min and max scaling between 0 and 1. I was able to understand that my model was overfitting the data by observing the difference between the training and testing accuracies. I thought I could improve the results by decreasing the regularization parameter "C", to force the model to become simpler. This did improve the testing accuracy up to about 88%, but I was surprised that when I increased the regularization parameter (allowing the model to become a lot more complicated) the testing accuracy increased to 93%. I did not expect that result. I would love to test the model on some new validation data.

The support vector machine did achieve a greater accuracy than the perceptron I used in notebook 4 (85.7%). This is promising, but I still think with more exploration and time to explore the options I have stated above, the accuracy could increase.